In [ ]:
import os
from datetime import date

#from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly.graph_objects as go

import pandas as pd

In [ ]:
#JupyterDash.infer_jupyter_proxy_config()

In [ ]:
"""
Loading dataset
---------------

Title: Population by month, city district, gender, age group and origin, since 1998

Location: Zürich city

Link: https://data.stadt-zuerich.ch/dataset/bev_monat_bestand_quartier_geschl_ag_herkunft_od3250
"""
ROOT_PATH = os.getcwd()
PATH = os.path.join(ROOT_PATH, "inputs", "zuerich_pop_stats.csv")

df = pd.read_csv(PATH, delimiter=",")

COL_RENAME = {
    "StichtagDatJahr": "year",
    "StichtagDatMonat": "month",
    "KreisLang": "district",
    "AnzBestWir": "pop_cnt",
    "SexLang": "sex",
    "HerkunftLang": "foreigner"
}
df = df.rename(columns=COL_RENAME)
df.head()

In [ ]:
# Add semester indicator
df["semester"] = df["StichtagDatMM"].apply(lambda x: "Semester 1" if x <= 6 else "Semester 2")

In [ ]:
# Aggregation per sex
GROUPBY = ['year', 'semester', 'month', 'StichtagDatMM', 'sex']
df_sex = df.groupby(GROUPBY).pop_cnt.agg('sum')
df_sex = df_sex.to_frame().reset_index().sort_values(by=['StichtagDatMM'])
df_sex.head()

In [ ]:
# Aggregation per swiss
GROUPBY = ['year', 'semester', 'month', 'StichtagDatMM', 'foreigner', 'district']
df_foreigner = df.groupby(GROUPBY).pop_cnt.agg('sum')
df_foreigner = df_foreigner.to_frame().reset_index()

df_for = df_foreigner[df_foreigner["foreigner"] == "Ausländer/in"].rename(columns={"pop_cnt": "foreign_pop_cnt"})
df_swiss = df_foreigner[df_foreigner["foreigner"] == "Schweizer/in"].rename(columns={"pop_cnt": "swiss_pop_cnt"})
df_swiss = df_swiss[["year", "month", "swiss_pop_cnt", "district"]]

df_for = pd.merge(
    df_for,
    df_swiss,
    how="inner",
    on=['year', 'month', 'district'],
)

df_for["ratio_for"] = df_for["foreign_pop_cnt"] / (df_for["swiss_pop_cnt"] + df_for["foreign_pop_cnt"])

df_for = df_for.sort_values(by=['district', 'year', 'StichtagDatMM'])
df_for["change"] = (df_for.ratio_for - df_for.ratio_for.shift(1))/df_for.ratio_for.shift(1)
df_for.head()

In [ ]:
# User profile values prepa
today = date.today()
cur_date = today.strftime("%B %d, %Y")

In [ ]:
# Dropdown items
YITEMS = df_sex["year"].unique().tolist()
YITEMS.sort(reverse=True)
YITEMS = [{'label': p, 'value': p} for p in YITEMS]

SITEMS = df_sex["semester"].unique()
SITEMS = [{'label': p, 'value': p} for p in SITEMS]

_DITEMS = df_for["district"].unique().tolist()
DITEMS = [{'label': p, 'value': p} for p in _DITEMS]

In [ ]:
# Creating the app layout
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

app.layout = html.Div([
    
    html.Div([
        html.Div([
            html.H4('Zürich Population Overview', style={'fontFamily': 'arial', 'paddingLeft': '10px', 'fontWeight': 'bold'})
        ], className='six columns'),
    
    html.Div([
        html.Div([
            html.H4('Exploratory Data Analysis', style={'textAlign': 'right', 'fontFamily': 'arial', 'paddingRight': '10px', 'fontWeight': 'bold'})
        ], className='six columns'),
        
    ], className='row', style={'backgroundColor': '#E1DCCA'})
    ]),
    
    
    html.Div([
        html.Div([
            dcc.Dropdown(id='district-dd', options=DITEMS, value="Kreis 2")
        ], className='four columns'),
        html.Div([
            dcc.Dropdown(id='year-dd', options=YITEMS, value=2021)
        ], className='four columns'),
        html.Div([
            dcc.Dropdown(id='semester-dd', options=SITEMS, value='Semester 1')
        ], className='four columns'),
    ], className='row'),
    
    
    html.Div([
        html.Div([
            dcc.Graph(id='g_barchart')
        ], className='six columns'),
    
    html.Div([
        html.Div([
            dcc.Graph(id='g_linechart')
        ], className='six columns'),
        
    ], className='row')
    ]),
    
    
    html.Div([
        html.Div([
            html.Footer('Current date:  {}'.format(cur_date), style={'fontFamily': 'arial', 'paddingLeft': '10px'})
        ], className='six columns'),
    
    html.Div([
        html.Div([
            html.Footer('Jonaddo Github', style={'textAlign': 'right', 'fontFamily': 'arial', 'paddingRight': '10px'})
        ], className='six columns'),
        
    ], className='row')
    ])
])

In [ ]:
# Bar chart
@app.callback(
    Output(component_id='g_barchart', component_property='figure'),
    
    Input(component_id='year-dd', component_property='value'),
    Input(component_id='semester-dd', component_property='value'),
    Input(component_id='district-dd', component_property='value'),
)
def update_barchart(y, s, d):
    mask = (df_for["year"] == y) & (df_for["semester"] == s) & (df_for["district"] == d)
    bc_filtered = df_for[mask]
    months = bc_filtered["month"].drop_duplicates()
    
    TITLE = {
        'text': 'Monthly percent change in foreigner ratio',
        'x': 0.5,
        'y': 0.9,
        'font_family': 'arial',
        'xanchor': 'center',
        'yanchor': 'top'
    }
    
    BARS = [go.Bar(x=bc_filtered[bc_filtered["district"] == DI]["change"], y=months, orientation='h', marker={'color': '#373A36'}, name=DI) for DI in _DITEMS]
    go_barchart = go.Figure(data=BARS)
    
    go_barchart.update_layout(barmode='relative', plot_bgcolor='white', height=400, title=TITLE, xaxis_tickformat=".2%")
    
    return go_barchart

In [ ]:
# Line chart
@app.callback(
    Output(component_id='g_linechart', component_property='figure'),
    
    Input(component_id='year-dd', component_property='value'),
    Input(component_id='semester-dd', component_property='value')
)
def update_linechart(y, s):
    mask = (df_sex["year"] == y) & (df_sex["semester"] == s)
    df_filtered = df_sex[mask]
    df_men = df_filtered[df_filtered["sex"] == "männlich"]
    df_women = df_filtered[df_filtered["sex"] == "weiblich"]
    months = df_filtered["month"].drop_duplicates()
    
    TITLE = {
        'text': 'Head count over time per sex',
        'x': 0.5,
        'y': 0.9,
        'font_family': 'arial',
        'xanchor': 'center',
        'yanchor': 'top'
    }
    
    go_linechart = go.Figure(data=[
        go.Scatter(x=months, y=df_men["pop_cnt"], marker={'color': '#373A36'}, name='Men'),
        go.Scatter(x=months, y=df_women["pop_cnt"], marker={'color': '#ADDCE6'}, name='Women'),
    ])
    
    go_linechart.update_layout(plot_bgcolor='white', height=400, title=TITLE)
    return go_linechart

In [ ]:
## Run app
'''
Port numbers are assigned in various ways, based on three ranges:
 1. System Ports (0-1023)
 2. User Ports (1024-49151)
 3. Dynamic and/or Private Ports (49152-65535)
 
Source: https://www.iana.org/assignments/service-names-port-numbers/service-names-port-numbers.xhtml
'''
MY_HOST = '127.0.0.1'
MY_PORT = '8050'

app.run_server(host=MY_HOST, port=MY_PORT)